In [9]:
import numpy as np

# טען את הקבצים המקוריים של התמונות והלייבלים
images = np.load(r"C:\Users\guymk\Downloads\images (1).npy")  # החלף בשם הקובץ המקורי שלך
labels = np.load(r"C:\Users\guymk\Downloads\labels (1).npy")  # החלף בשם הקובץ המקורי שלך

# בחר 1000 תמונות אקראיות להקטנה
indices = np.random.choice(len(images), 1000, replace=False)
images_small = images[indices]
labels_small = labels[indices]

print('New small dataset shape:', images_small.shape, labels_small.shape)


New small dataset shape: (1000, 224, 224, 3) (1000,)


In [11]:
import numpy as np

np.save('images_1000.npy', images_small)
np.save('labels_1000.npy', labels_small)

print('Files saved successfully as images_1000.npy and labels_1000.npy')


Files saved successfully as images_1000.npy and labels_1000.npy


In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    images_small, labels_small, test_size=0.2, random_state=42)


In [17]:
from tensorflow.keras.utils import to_categorical

num_classes = 196
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

print('X_train:', X_train.shape)
print('y_train:', y_train_encoded.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test_encoded.shape)


X_train: (800, 224, 224, 3)
y_train: (800, 196)
X_test: (200, 224, 224, 3)
y_test: (200, 196)


In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# פיצול הנתונים ל-80% אימון ו-20% מבחן (ללא stratify)
X_train, X_test, y_train, y_test = train_test_split(
    images_small, labels_small, test_size=0.2, random_state=42)

# קידוד One-hot של התוויות
num_classes = 196
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

print('X_train:', X_train.shape)
print('y_train:', y_train_encoded.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test_encoded.shape)


X_train: (800, 224, 224, 3)
y_train: (800, 196)
X_test: (200, 224, 224, 3)
y_test: (200, 196)


In [21]:
import numpy as np

# טעינת קובצי ה-1000 תמונות והלייבלים
images = np.load('images_1000.npy')
labels = np.load('labels_1000.npy')

print('Loaded dataset shape:', images.shape, labels.shape)


Loaded dataset shape: (1000, 224, 224, 3) (1000,)


In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# פיצול הנתונים ל-80% אימון ו-20% מבחן
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42)  # ללא stratify

# קידוד One-hot של הלייבלים
num_classes = 196
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

print('X_train:', X_train.shape)
print('y_train:', y_train_encoded.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test_encoded.shape)


X_train: (800, 224, 224, 3)
y_train: (800, 196)
X_test: (200, 224, 224, 3)
y_test: (200, 196)


In [27]:
from tensorflow.keras.utils import to_categorical

# קידוד היעדים ל-One-Hot
num_classes = 196
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

print('y_train shape after encoding:', y_train_encoded.shape)
print('y_test shape after encoding:', y_test_encoded.shape)


y_train shape after encoding: (800, 196)
y_test shape after encoding: (200, 196)


## מודל אימון 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

# 🟢 1. Data Augmentation (שיפור הנתונים לפני אימון)
datagen = ImageDataGenerator(
    rotation_range=20,  # סיבובים עד 20 מעלות
    width_shift_range=0.2,  # הזזה אופקית של 20%
    height_shift_range=0.2,  # הזזה אנכית של 20%
    horizontal_flip=True,  # היפוך תמונות
    zoom_range=0.2,  # זום אקראי עד 20%
    shear_range=0.2,  # שיפוע תמונות
    fill_mode='nearest'  # השלמת פיקסלים חסרים
)

# 🟢 2. בניית המודל עם יותר שכבות
model = models.Sequential([
    # שכבה ראשונה: 64 פילטרים במקום 32
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),  # מניעת Overfitting

    # שכבה שנייה: 128 פילטרים
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    # שכבה שלישית: 256 פילטרים
    layers.Conv2D(256, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    # שכבה רביעית: 512 פילטרים (ללמידה עמוקה יותר)
    layers.Conv2D(512, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),

    # סיום עם גלובל אברג'
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(196, activation='softmax')  # 196 מחלקות של רכבים
])

# 🟢 3. קומפילציה עם אופטימיזציה מתקדמת (SGD במקום Adam)
model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# הצגת המבנה
model.summary()

# 🟢 4. הכנת ה-Data Augmentation
train_generator = datagen.flow(X_train, y_train_encoded, batch_size=16)

# 🟢 5. אימון המודל עם יותר Epochs ושימוש ב-Validation Set
history = model.fit(
    train_generator,
    epochs=30,  # יותר epochs ללמידה טובה יותר
    validation_data=(X_test, y_test_encoded),
    batch_size=16)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 222, 222, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 222, 222, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 220, 220, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 220, 220, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 110, 110, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 110, 110, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 108, 108, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 108, 108, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 106, 106, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 106, 106, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 53, 53, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 53, 53, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 51, 51, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 51, 51, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 49, 49, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 49, 49, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 24, 24, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 24, 24, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 5,056,260 (19.29 MB)

 Trainable params: 5,052,420 (19.27 MB)

 Non-trainable params: 3,840 (15.00 KB)

C:\Users\guymk\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
41/50 ━━━━━━━━━━━━━━━━━━━━ 2:16 15s/step - accuracy: 0.0029 - loss: 5.8043

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 🟢 1. טעינת התמונות והתגיות
X = np.load("C:/Users/guymk/images_1000.npy")
y = np.load("C:/Users/guymk/labels_1000.npy")  # תוודא שיש לך את הקובץ הזה

# 🟢 2. נרמול התמונות וקידוד המחלקות
X = X.astype('float32') / 255.0
y_encoded = to_categorical(y, num_classes=196)

# 🟢 3. פיצול ל-Train/Test בלי stratify כדי למנוע בעיה עם מחלקות נדירות
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42)

# 🟢 4. הגדרת Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

train_generator = datagen.flow(X_train, y_train_encoded, batch_size=16)
val_generator = ImageDataGenerator().flow(X_test, y_test_encoded, batch_size=16)

# 🟢 5. בניית המודל
model = models.Sequential([
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.3),

    layers.Conv2D(512, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),

    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(196, activation='softmax')
])

# 🟢 6. קומפילציה
model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 🟢 7. אימון
history = model.fit(
    train_generator,
    epochs=20,  # אפשר להתחיל עם פחות מ-30 כדי לבדוק מהר
    validation_data=val_generator
)

# 🟢 8. שלב הטסט – הערכת ביצועים
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"🔍 Test Accuracy: {test_accuracy:.4f}")
print(f"📉 Test Loss: {test_loss:.4f}")

C:\Users\guymk\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\guymk\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 803s 16s/step - accuracy: 0.0029 - loss: 5.7487 - val_accuracy: 0.0000e+00 - val_loss: 9.5492
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1066s 21s/step - accuracy: 0.0221 - loss: 5.3412 - val_accuracy: 0.0000e+00 - val_loss: 7.0745
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 297s 6s/step - accuracy: 0.0256 - loss: 5.1838 - val_accuracy: 0.0050 - val_loss: 6.1452
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 248s 5s/step - accuracy: 0.0130 - loss: 5.1652 - val_accuracy: 0.0000e+00 - val_loss: 10.9745
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 3218s 66s/step - accuracy: 0.0266 - loss: 5.0762 - val_accuracy: 0.0000e+00 - val_loss: 8.3856
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 2070s 42s/step - accuracy: 0.0194 - loss: 5.0180 - val_accuracy: 0.0000e+00 - val_loss: 10.2842
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 6800s 137s/step - accuracy: 0.0142 - loss: 5.0434 - val_accuracy: 0.0000e+00 - val_loss: 12.1471
Epoch 8/20
33/50 ━━━━━━━━━━━━━━━━━━━━ 27:24 97s/step - accuracy: 0.0391 - 